# Scrapy的下载中间件详解
　　参考：[官方文档](https://docs.scrapy.org/en/2.0/topics/settings.html?highlight=DOWNLOADER_MIDDLEWARES#downloader-middlewares-base)<br>
　　　　　[Scrapy爬虫之下载中间件的处理流程](https://blog.csdn.net/qq_35556064/article/details/91346493)<br>
　　　　　[一些常用中间件编写](https://www.jianshu.com/p/a91c663579eb)<br>
　　　　　[彻底搞懂scrapy中间件](https://www.pianshen.com/article/8549378131/)<br>
　　　　　[middlewares下载中间件、断点爬取、设置文件参数](https://www.jianshu.com/p/a91c663579eb)<br>
　　　　　[scrapy启动流程源码分析(6)Downloader下载器](https://blog.csdn.net/csdn_yym/article/details/85721661)<br>
　　　　　[scrapy源码9：middleware的源码分析](https://cuiyonghua.blog.csdn.net/article/details/110386947)<br>
　　　　　[scrapy源码7：downloader的源码分析](https://cuiyonghua.blog.csdn.net/article/details/108335043)<br>
　　　　　[Scrapy的抓取流程—Downloader](https://blog.csdn.net/okm6666/article/details/89575552)<br>
　　　　　[Downloader 中间件（Downloader Middleware）详解](https://blog.csdn.net/qq_20288327/article/details/113524585)<br>
　　　　　[scrapy源码分析（十一）----------下载器Downloader](https://blog.csdn.net/happyanger6/article/details/53572072/)<br>
　　　　　[彻底搞懂Scrapy的中间件（一）](https://www.cnblogs.com/xieqiankun/p/know_middleware_of_scrapy_1.html)<br>
　　　　　[Scrapy进阶知识点总结（六）——中间件详解](https://www.cnblogs.com/fengf233/p/11453375.html)<br>
　　　　　[学习笔记 2-5 Scrapy的中间件](https://blog.csdn.net/kissazhu/article/details/80865739)<br>


## 一、Downloader下载器
　　Downloader包含了从调度器调取url之后到scraper获取返回的网页内容之前的所有步骤，关系到网页如何下载，网络通信/HTTP协议/服务器等一系列知识，是最复杂的一部分内容。<br>
### 1.1、Downloader初始化
　　在之前Crawler.crawl()创建ExecutionEngine执行引擎的时候，就已经初始化了Downloader对象。此对象在配置中定义，默认为（scrapy/settings/default_settings.py）：<br>
>DOWNLOADER = 'scrapy.core.downloader.Downloader'<br>

　　关键对象有4个:slots,active,DownloadHandlers,DownloaderMiddlewareManager以及一些配置选项。<br>
### 1.2、slots对象
　　1、这个slots是一个存储Slot对象的字典，key是request对应的域名，值是一个Slot对象。<br>
　　2、Slot对象用来控制一种Request下载请求，通常这种下载请求是对于同一个域名。<br>
　　3、这个Slot对象还控制了访问这个域名的并发度，下载延迟控制，随机延时等，主要是为了控制对一个域名的访问策略，一定程度上避免流量过大被封IP。<br>

### 1.3、active对象
　　active是一个活动集合，用于记录当前正在下载的request集合。<br>

### 1.4、DownloadHandlers对象
　　是一个DownloadHandlers对象，它控制了许多handlers,对于不同的下载协议使用不同的handlers。<br>
　　后面下载网页会调用handler的download_request方法。<br>
　　默认支持handlers如下（scrapy/settings/default_settings.py）:<br>
>DOWNLOAD_HANDLERS_BASE = {<br>
　　'data': 'scrapy.core.downloader.handlers.datauri.DataURIDownloadHandler',<br>
　　'file': 'scrapy.core.downloader.handlers.file.FileDownloadHandler',<br>
　　'http': 'scrapy.core.downloader.handlers.http.HTTPDownloadHandler',<br>
　　'https': 'scrapy.core.downloader.handlers.http.HTTPDownloadHandler',<br>
　　's3': 'scrapy.core.downloader.handlers.s3.S3DownloadHandler',<br>
　　'ftp': 'scrapy.core.downloader.handlers.ftp.FTPDownloadHandler',<br>
}<br>

### 1.5、DownloaderMiddlewareManager对象
　　Downloader的中间件实现与scraper中间件的实现过程一样，也是通过Manager类管理。<br>
　　默认中间件（scrapy/settings/default_settings.py）：<br>
>DOWNLOADER_MIDDLEWARES_BASE = {<br>
　　\# Engine side<br>
　　'scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware': 100,<br>
　　'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware': 300,<br>
　　'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware': 350,<br>
　　'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware': 400,<br>
　　'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': 500,<br>
　　'scrapy.downloadermiddlewares.retry.RetryMiddleware': 550,<br>
　　'scrapy.downloadermiddlewares.ajaxcrawl.AjaxCrawlMiddleware': 560,<br>
　　'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware': 580,<br>
　　'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware': 590,<br>
　　'scrapy.downloadermiddlewares.redirect.RedirectMiddleware': 600,<br>
　　'scrapy.downloadermiddlewares.cookies.CookiesMiddleware': 700,<br>
　　'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware': 750,<br>
　　'scrapy.downloadermiddlewares.stats.DownloaderStats': 850,<br>
　　'scrapy.downloadermiddlewares.httpcache.HttpCacheMiddleware': 900,<br>
　　\# Downloader side<br>
}<br>


## 二、Downloader源码解析


In [ ]:
# scrapy.core.downloader.__init__.py
from __future__ import absolute_import
# 建议尽可能多的使用绝对导入，因此在你的代码中使用from pkg improt string是适宜的。

import random
import warnings     # 这就是导入随机数和警告
from time import time
from datetime import datetime
from collections import deque     # 这些也是导入基本的包， 没啥问题。 

import six
from twisted.internet import reactor, defer, task
# 这里导入了兼容包six， 以及twisted 下的reactor反应堆， defer以及task。
from scrapy.utils.defer import mustbe_deferred    # 导入强制deferred 
from scrapy.utils.httpobj import urlparse_cached   # 这个我们定位过去看看
from scrapy.resolver import dnscache    # dnscache = LocalCache(10000) ，说明dnscache 是个dict 的。 
from scrapy import signals              # 导入信号
from .middleware import DownloaderMiddlewareManager
from .handlers import DownloadHandlers    # 导入了下载中间件和下载处理器


_urlparse_cache = weakref.WeakKeyDictionary()
def urlparse_cached(request_or_response):
    """Return urlparse.urlparse caching the result, where the argument can be a
    Request or Response object
    方法是返回一个结果urlparse,urlparse的缓存结果。 参数可以是请求也可以是响应。 
    如果对象不再字典里面， 就缓存下， 然后返回。
    """
    if request_or_response not in _urlparse_cache:
        _urlparse_cache[request_or_response] = urlparse(request_or_response.url)
    return _urlparse_cache[request_or_response]


class Slot(object):
    """Downloader slot"""

    def __init__(self, concurrency, delay, randomize_delay):
        # 初始化工作 。 并发，延迟， 是否随机延迟， 活动集合， 队列， 正在传输集合， 最后调用， 
        self.concurrency = concurrency
        self.delay = delay
        self.randomize_delay = randomize_delay   # 是否开启随机延迟

        self.active = set()
        self.queue = deque()
        self.transferring = set()
        self.lastseen = 0
        self.latercall = None

    def free_transfer_slots(self):
        # 返回：空闲的传输槽 = 并发量 - 当前传输的个数
        return self.concurrency - len(self.transferring)


    def download_delay(self):
        # 下载延迟，如果随机延迟开启的话 在0.5 ~ 1.5倍 delay 之间返回一个值，否则直接使用delay值。 
        if self.randomize_delay:
            return random.uniform(0.5 * self.delay, 1.5 * self.delay)
        return self.delay

    def close(self):
        # 关闭下载器，如果最后一个调用存在，代用cancel取消。
        if self.latercall and self.latercall.active():
            self.latercall.cancel()

    def __repr__(self):
        cls_name = self.__class__.__name__
        return "%s(concurrency=%r, delay=%0.2f, randomize_delay=%r)" % (
            cls_name, self.concurrency, self.delay, self.randomize_delay)

    def __str__(self):
        return (
            "<downloader.Slot concurrency=%r delay=%0.2f randomize_delay=%r "
            "len(active)=%d len(queue)=%d len(transferring)=%d lastseen=%s>" % (
                self.concurrency, self.delay, self.randomize_delay,
                len(self.active), len(self.queue), len(self.transferring),
                datetime.fromtimestamp(self.lastseen).isoformat()
            )
        )
    # 这2个就是基本的方法了。 没啥问题的。 

    
def _get_concurrency_delay(concurrency, spider, settings):
    # 获取并发的延迟，从设置里面获取download_delay 也就是获取下载延迟设置。 
    # 如果爬虫有对应的下载延迟设置DOWNLOAD_DELAY，提示警告（已废弃）。并采用该延迟设置值。
    # 如果爬虫有对应的download_delay设置，设置延迟值。
    # 如果有最大并发请求个数，获取该最大并发数
    # 最终返回 （最大并发数，延迟量） 
    delay = settings.getfloat('DOWNLOAD_DELAY')
    if hasattr(spider, 'DOWNLOAD_DELAY'):
        warnings.warn("%s.DOWNLOAD_DELAY attribute is deprecated, use %s.download_delay instead" %
                      (type(spider).__name__, type(spider).__name__))
        delay = spider.DOWNLOAD_DELAY
    if hasattr(spider, 'download_delay'):
        delay = spider.download_delay

    if hasattr(spider, 'max_concurrent_requests'):
        concurrency = spider.max_concurrent_requests

    return concurrency, delay


class Downloader(object):

    DOWNLOAD_SLOT = 'download_slot'

    def __init__(self, crawler):
        # 这个是downloader的初始化方法， 根据一个crawler去初始化，
        self.settings = crawler.settings
        self.signals = crawler.signals    # 获取crawler的设置和信号
        self.slots = {}
        self.active = set()    # 构造一个slots字典，和一个活动集合（正在并发请求的任务集合，用于限制并发数）。 
        self.handlers = DownloadHandlers(crawler)    # 根据crawler构造一个下载处理对象。
        self.total_concurrency = self.settings.getint('CONCURRENT_REQUESTS')
        self.domain_concurrency = self.settings.getint('CONCURRENT_REQUESTS_PER_DOMAIN')
        self.ip_concurrency = self.settings.getint('CONCURRENT_REQUESTS_PER_IP')
        self.randomize_delay = self.settings.getbool('RANDOMIZE_DOWNLOAD_DELAY')
        self.middleware = DownloaderMiddlewareManager.from_crawler(crawler)
        # 从设置获取并发请求数，每个域名的最大请求数，每个ip的并发请求限制，随机下载延迟，构造中间件管理器。 
        self._slot_gc_loop = task.LoopingCall(self._slot_gc)    # 是一个定时的循环
        self._slot_gc_loop.start(60)   # # 这2句话的意思就是每60s就去执行下self._slot_gc方法（打印slot信息）。

    def fetch(self, request, spider):
        # 活动集合添加请求，下载中间件完成下载获取deffered对象， 添加一个回调方法移除请求。
        def _deactivate(response):
            self.active.remove(request)
            return response

        self.active.add(request)
        dfd = self.middleware.download(self._enqueue_request, request, spider)    # 走下载器中间件
        return dfd.addBoth(_deactivate)

    def needs_backout(self):
        # 判断当前活动的个数是不是超出了总的并发量
        return len(self.active) >= self.total_concurrency

    def _get_slot(self, request, spider):
        # 获取 key -> 请求域名
        # 如果key不在slots里面：如果ip并发限制了，就使用ip限制，否则使用域名并发限制。 
        # 从设置里面获取并发和延迟，并根据key创建一个slot
        # 返回key 和 slot
        key = self._get_slot_key(request, spider)
        if key not in self.slots:
            conc = self.ip_concurrency if self.ip_concurrency else self.domain_concurrency
            conc, delay = _get_concurrency_delay(conc, spider, self.settings)
            self.slots[key] = Slot(conc, delay, self.randomize_delay)

        return key, self.slots[key]

    def _get_slot_key(self, request, spider):
        # 这个先看看meta有没有download_slot ，如果有的话就直接返回
        # 没有话，就从urlparse缓冲里面获取到主机名，如果为空，就设置为‘’ 
        # 如果有ip并发限制，从dns缓冲中获取
        # 返回的 download_slot  是一个请求的 域名
        if 'download_slot' in request.meta:
            return request.meta['download_slot']

        key = urlparse_cached(request).hostname or ''
        if self.ip_concurrency:
            key = dnscache.get(key, key)

        return key 

    def _enqueue_request(self, request, spider):
        # 根据 request 和 spider 获取 key、slot , 设置请求的meta。
        # slot的active集合添加请求，添加回调方法去移除active中完成了的请求request。
        # 发送一个信号量： 请求到达下载器
        # slot的队列添加一个 request、deferred。
        # 调用_process_queue 去处理队列。 返回deferred对象。
        key, slot = self._get_slot(request, spider)
        request.meta['download_slot'] = key

        def _deactivate(response):
            slot.active.remove(request)
            return response

        slot.active.add(request)
        self.signals.send_catch_log(signal=signals.request_reached_downloader,
                                    request=request,spider=spider)
        deferred = defer.Deferred().addBoth(_deactivate)
        slot.queue.append((request, deferred))
        self._process_queue(spider, slot)
        return deferred

    def _process_queue(self, spider, slot):
        # 处理队列的方法，如果最后一个调用在，就返回， 
        # 获取当前时间，从slot获取下载延迟。如果有延迟，slot记录的上次的lastseen + 延迟时间 - now 得到
        # 上次使用距离当前的时间差值，如果大于0 说明等待时间小于延迟时间了，可以进行下一步操作了。 
        # 设置下lastcall 设置下几秒之后执行方法。 也就是设置等待几秒去执行self._process_queue 方法。

        # 如果还有空闲的传输clost的话：
        # 设置lastseen为now,并从队列取出一个请求，调用_download返回一个deferred对象，添加到deferred链上，
        # 如果有延迟，递归执行 _process_queue 方法。
        if slot.latercall and slot.latercall.active():
            return

        # Delay queue processing if a download_delay is configured
        now = time()
        delay = slot.download_delay()
        if delay:
            penalty = delay - now + slot.lastseen
            if penalty > 0:
                slot.latercall = reactor.callLater(penalty, self._process_queue, spider, slot)
                return
        """
        在process_request部分的最后，会将下载器中间件下载前，方法返回的request重新扔回给downloader进行下载，
        _enqueue_request方法中将request加入属于downloader的slot对象的队列中，然后对这个队列进行消费。如
        果配置文件中设置了DOWNLOAD_DELAY，则会在延迟时间之后再重新调用_process_queue。
        _process_queue方法中，从slot的queue中取出request，调用_download方法对该请求进行下载。
        使用DownloadHandlers的实例对request进行下载。下载结束后，从slot的transferring集合中取出。（这里
        是为了配合配置文件中的同时下载数使用）
        """
        # Process enqueued requests if there are free slots to transfer for this slot
        while slot.queue and slot.free_transfer_slots() > 0:
            slot.lastseen = now
            request, deferred = slot.queue.popleft()
            dfd = self._download(slot, request, spider)
            dfd.chainDeferred(deferred)
            # prevent burst if inter-request delays were configured
            if delay:
                self._process_queue(spider, slot)
                break

    def _download(self, slot, request, spider):
        # The order is very important for the following deferreds. Do not change!
        # 创建下载的deferred， 
        # 在查询任务队列获取下一次请求之前，通知响应下载器监听最近的下载。
        # 在接收到响应之后，将请求从下载状态中删除，以释放传输槽（slot），以便之后的请求任务使用（包括
        # 来自下载中间件的请求）。
        # 1. Create the download deferred
        dfd = mustbe_deferred(self.handlers.download_request, request, spider)

        # 2. Notify response_downloaded listeners about the recent download
        # before querying queue for next request
        def _downloaded(response):
            self.signals.send_catch_log(signal=signals.response_downloaded,
                                        response=response,
                                        request=request,
                                        spider=spider)
            return response
        dfd.addCallback(_downloaded)

        # 3. After response arrives,  remove the request from transferring
        # state to free up the transferring slot so it can be used by the
        # following requests (perhaps those which came from the downloader
        # middleware itself)
        slot.transferring.add(request)

        def finish_transferring(_):
            slot.transferring.remove(request)
            self._process_queue(spider, slot)
            return _

        return dfd.addBoth(finish_transferring)

    def close(self):
        # 关闭方法，关闭gc_loop ，以及所有slot。
        self._slot_gc_loop.stop()
        for slot in six.itervalues(self.slots):
            slot.close()

    def _slot_gc(self, age=60):
        # 获取近一分钟前的时间
        # 遍历slots。如果slot的状态不是active,且 上次访问时间 + 延迟 < mintime的话（早于1分钟前），
        # 就从slots里面pop出来，并关闭。
        mintime = time() - age
        for key, slot in list(self.slots.items()):
            if not slot.active and slot.lastseen + slot.delay < mintime:
                self.slots.pop(key).close()





## 三、Middleware（下载中间件）使用
　　Download Middleware是Scrapy的请求/响应处理的钩子框架。它是一个轻量级的低级系统，用于全局改变Scrapy的请求和响应。常用来添加代理，添加cookie，失败重新发起请求等等。<br>

### 3.1、激活Download Middleware
　　要激活Download Middleware，请在settings.py中激活 **DOWNLOADER_MIDDLEWARES**设置，该设置是一个dict，其键是中间件类路径，其值是中间件处理顺序值。要禁用某个中间件，只需把值设为None即可。<br>
　　这是一个例子：
>DOWNLOADER_MIDDLEWARES = {<br>
    'myproject.middlewares.CustomDownloaderMiddleware': 543,<br>
}<br>

　　如果要禁用内置中间件DOWNLOADER_MIDDLEWARES_BASE默认定义和启用的中间件 ，则必须在项目的DOWNLOADER_MIDDLEWARES设置中定义它为None。例如，如果要禁用用户代理中间件：<br>
>DOWNLOADER_MIDDLEWARES = {<br>
    'myproject.middlewares.CustomDownloaderMiddleware': 543,<br>
    'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,<br>
}

### 3.2、download方法详解<br>
　　DownloaderMiddlewareManager的download方法，在内部实现了process_request、process_response、process_exception三个方法，用于调用各个中间件的同名方法对请求、响应、异常的处理。此三个方法的处理逻辑如下图：<br>
　　![下载中间件对请求、响应、异常的处理流程图](./images/scrapy_middleware流程图.png)
  
|方法|可返回类型|处理流程|
|:---|:---|:---|
|process_request(self, request, spider)|1、None<br>2、Response<br>3、Request<br>4、Raise IgnoreRequest|1、返回None的处理流程<br>2、返回Response的处理流程<br>3、返回Request的处理流程<br>4、返回Exception的处理流程|
|process_response(self, request, response,spider)|A、Response<br>B、Request<br>C、Raise IgnoreRequest|A、返回Response的处理流程<br>B、返回Request的处理流程<br>C、返回Exception的处理流程<br>|
|process_exception(self, request, exception, spider)|i、None<br>ii、Response<br>iii、Request|i、返回None的处理流程<br>ii、返回Response的处理流程<br>iii、返回Request的处理流程<br>|




### 3.2.1、process_request(self, request, spider)
　　返回值：<br>
　　1、返回None：Scrapy将请求传给后面的中间件的process_request函数继续处理此请求，直到最后。<br>
　　2、返回 Response 对象：Scrapy将不再调用其他process_request()或process_exception()方法，而是直接进入最后一个中间件（倒序处理response）的process_response函数里来处理该Response对象，所有已安装的中间件的process_response()方法都会被调用。<br>
　　3、返回 Request 对象：Scrapy将停止调用之后的process_request方法并重新处理返回的请求（既进入第一个中间件的process_request）。<br>
　　4、引发 IgnoreRequest 异常：已安装的下载中间件的process_exception()方法将被调用（从最后一个开始倒序调用）。如果它们都不处理异常，则调用request()(Request.errback)中的errback函数。如果没有代码处理引发的异常，则会忽略它并且不会记录。<br>
　　参数：<br>
　　request（Request对象） - 正在处理的请求<br>
　　spider（Spider对象） - 此请求所针对的蜘蛛<br>
### 3.2.2、process_response(self, request, response,spider)
　　返回值：<br>
　　1、返回 Response 对象：将响应交给下一个中间件的 process_response 继续处理。也就是对其他中间件没影响。<br>
　　2、返回 Request 对象：不再调用 process_response，而是将返回 request 重新加入到调度队列，进而从新从第一个中间件的process_request 处理该请求对象。<br>
　　3、引发 IgnoreRequest 异常：注意此时不会进入process_exception函数，而是调用Reuest对象的errorblack回调函数来处理，如果没有定义这个回调则忽略。（已安装的下载中间件的process_exception()方法将被调用。如果它们都不处理异常，则调用request()(Request.errback)中的errback函数。如果没有代码处理引发的异常，则会忽略它并且不会记录。） **不同解释，需考证**<br>
　　参数：<br>
　　request（Request对象） - 发起响应的请求<br>
　　response（Response对象） - 正在处理的响应<br>
　　spider（Spider对象） - 此响应所针对的蜘蛛<br>
### 3.2.3、process_exception(requset, exception, spider)
　　返回值：<br>
　　1、返回None，Scrapy将继续处理此异常，执行任何其他已安装中间件的process_exception()方法，直到没有剩下中间件并且默认异常处理开始。<br>
　　2、返回 Response 对象：Scrapy不再调用其他中间件的process_exception()方法，而是将响应交从最后一个中间件开始的process_response方法处理。<br>
　　3、返回 Request 对象：重新开始处理请求，即重新加入到调度队列，进而重新进入第一个中间件的process_request函数。这个用于失败重复调用时很有用。<br>
　　参数：<br>
　　request（是一个Request对象） - 生成异常的请求<br>
　　exception（一个Exception对象） - 引发的异常<br>
　　spider（Spider对象） - 此请求所针对的蜘蛛<br>
### 3.2.4 注意
　　中间件需要安装才能处理，请记得安装配置中间件的优先级。<br>
　　中间件的优先级：数字越小越靠近引擎，数字越大越靠近下载器。请求最先从引擎处开始传递，逐次经过优先级数字越来越大的中间件；相反，下载器返回响应时，则逐次经过优先级数字越来越小的中间件处理。因此，以上图为例：不要认为一个请求最先是中间件1处理的，那么响应也会最先由中间件1来处理，这是错误的。请求最先由优先级数字最小的中间件来处理，而响应最先由优先级数字最大的中间件处理。<br>



In [ ]:
# scrapy.core.downloader.middleware.py
import six  # 导入six兼容包
from twisted.internet import defer   # 导入defer
from scrapy.http import Request, Response    # 导入请求和响应
from scrapy.middleware import MiddlewareManager
from scrapy.utils.defer import mustbe_deferred    # 导入中间件管理 和mustbe_defferred
from scrapy.utils.conf import build_component_list    # 导入构建中间件列表函数


class DownloaderMiddlewareManager(MiddlewareManager):

    component_name = 'downloader middleware'

    @classmethod
    def _get_mwlist_from_settings(cls, settings):
        # 从设置里面获取下载中间件， 然后构建下载中间件列表
        return build_component_list(
            settings.getwithbase('DOWNLOADER_MIDDLEWARES'))
    
    def _add_middleware(self, mw):
        # 添加下载中间件，获取定义的几个方法，并添加到对应的列表中。 
        # 从此处可以看出，我们自定义下载中间件时，就是自定义process_request，process_response，process_exception。 
        if hasattr(mw, 'process_request'):
            self.methods['process_request'].append(mw.process_request)
        if hasattr(mw, 'process_response'):
            self.methods['process_response'].insert(0, mw.process_response)
        if hasattr(mw, 'process_exception'):
            self.methods['process_exception'].insert(0, mw.process_exception)

    def download(self, download_func, request, spider):
        # download里面定义了3个方法对应上面的三个方法：迭代调用各个列表中的对应方法。 
        @defer.inlineCallbacks
        def process_request(request):
            # 遍历 process_request 的方法列表（获取每个中间件的对应方法）。
            # 获取对应方法处理的响应，断言响应不为none,response,request,抛出异常。
            # 如果响应不为空的话， 就直接defer.returnValue。
            # 遍历完毕方法链之后，使用download_func获取结果。
            '''如果在某个Middleware中间件的process_request中处理完之后，生成了一个response对象，那么会直接
            将这个response return 出去，跳出循环，不再处理其他的process_request，之前我们的header，proxy中
            间件，都只是加个user-agent，加个proxy，并不做任何return值。还需要注意一点:就是这个return的必须
            是Response对象。'''
            for method in self.methods['process_request']:
                response = yield method(request=request, spider=spider)
                assert response is None or isinstance(response, (Response, Request)), \
                        'Middleware %s.process_request must return None, Response or Request, got %s' % \
                        (six.get_method_self(method).__class__.__name__, response.__class__.__name__)
                if response:
                    defer.returnValue(response)
            '''如果在上面的所有process_request中，都没有返回任何Response对象的话,最后会将这个加工过的Request送
            往download_func，进行下载，返回的就是一个Response对象,然后依次经过各个Middleware中间件的
            process_response方法进行加工'''
            defer.returnValue((yield download_func(request=request,spider=spider))) # 回调_enqueue_request传入request

        @defer.inlineCallbacks
        def process_response(response):
            # 断言响应为none,抛出异常。
            # 判定响应，如果响应是请求，调用defer.returnValue处理。
            # 遍历 process_response 的方法列表（获取每个中间件的对应方法）。
            # 断言响应结果非 Response or Request， 抛出异常。 如果响应是request的话，调用defer.returnValue处理。
            # 最后使用defer.returnValue处理响应。 
            assert response is not None, 'Received None in process_response'
            if isinstance(response, Request):
                defer.returnValue(response)

            for method in self.methods['process_response']:
                response = yield method(request=request, response=response,
                                        spider=spider)
                assert isinstance(response, (Response, Request)), \
                    'Middleware %s.process_response must return Response or Request, got %s' % \
                    (six.get_method_self(method).__class__.__name__, type(response))
                if isinstance(response, Request):
                    defer.returnValue(response)
            defer.returnValue(response)

        @defer.inlineCallbacks
        def process_exception(_failure):
            # 遍历 process_exception 的方法列表（获取每个中间件的对应方法），处理异常。
            # 断言响应结果非 Response or Request，抛出异常。
            # 返回失败值。
            exception = _failure.value
            for method in self.methods['process_exception']:
                response = yield method(request=request, exception=exception,
                                        spider=spider)
                assert response is None or isinstance(response, (Response, Request)), \
                    'Middleware %s.process_exception must return None, Response or Request, got %s' % \
                    (six.get_method_self(method).__class__.__name__, type(response))
                if response:
                    defer.returnValue(response)
            defer.returnValue(_failure)


